In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage
import warnings
warnings.filterwarnings('ignore')

# 1. Загружаем данные
df = pd.read_excel("energy_consumption.xlsx", header=1)
df = df.iloc[:, :35]

# 2. Отбираем только летние сутки (все столбцы, содержащие "summer")
summer_cols = [col for col in df.columns if 'summer' in col.lower()]
X_raw = df[summer_cols].copy()

# Убедимся, что нет пропусков
X_raw = X_raw.dropna()

print(f"Найдено летних суток: {len(summer_cols)}")
print(f"Форма данных (часы × сутки): {X_raw.shape}")

# 3. Транспонируем: теперь строки — сутки, столбцы — часы (0–23)
X_transposed = X_raw.T  # Это и есть наши объекты для кластеризации!
X_transposed.columns = [f'hour_{i}' for i in range(24)]  # переименуем часы для красоты

# 4. Стандартизация (ОЧЕНЬ ВАЖНО для временных рядов!)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_transposed)

# =============================================
# 5. Подбор числа кластеров (Elbow + Silhouette)
# =============================================
inertias = []
silhouettes = []
K = range(2, 15)

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouettes.append(silhouette_score(X_scaled, labels))

# Графики для выбора k
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ax[0].plot(K, inertias, 'bo-')
ax[0].set_xlabel('Число кластеров')
ax[0].set_ylabel('Инерция')
ax[0].set_title('Elbow Method')

ax[1].plot(K, silhouettes, 'ro-')
ax[1].set_xlabel('Число кластеров')
ax[1].set_ylabel('Silhouette Score')
ax[1].set_title('Silhouette Score')
plt.tight_layout()
plt.show()

# =============================================
# 6. K-means с оптимальным k (допустим, вы выбрали 4)
# =============================================
best_k = 4
kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
labels_kmeans = kmeans.fit_predict(X_scaled)

# =============================================
# 7. Иерархическая кластеризация
# =============================================
linked = linkage(X_scaled, method='ward')

plt.figure(figsize=(12, 6))
dendrogram(linked, truncate_mode='level', p=5)
plt.title('Дендрограмма (иерархическая кластеризация)')
plt.xlabel('Сутки')
plt.ylabel('Расстояние')
plt.show()

hier = AgglomerativeClustering(n_clusters=best_k, linkage='ward')
labels_hier = hier.fit_predict(X_scaled)

# =============================================
# 8. DBSCAN (подбор eps)
# =============================================
db = DBSCAN(eps=3.5, min_samples=3)  # eps подбирается по графику ниже
labels_db = db.fit_predict(X_scaled)

# Подбор eps
from sklearn.neighbors import NearestNeighbors

neighbors = NearestNeighbors(n_neighbors=4)
neighbors_fit = neighbors.fit(X_scaled)
distances, indices = neighbors_fit.kneighbors(X_scaled)
distances = np.sort(distances[:, 3], axis=0)

plt.figure(figsize=(8, 5))
plt.plot(distances)
plt.title('k-distance график для подбора eps (DBSCAN)')
plt.ylabel('Расстояние до 4-го соседа')
plt.xlabel('Объекты')
plt.grid(True)
plt.show()

# =============================================
# 9. ГЛАВНАЯ ВИЗУАЛИЗАЦИЯ: средние профили по кластерам
# =============================================
X_vis = X_transposed.copy()
X_vis['cluster_kmeans'] = labels_kmeans

plt.figure(figsize=(12, 6))
for cluster in sorted(X_vis['cluster_kmeans'].unique()):
    cluster_data = X_vis[X_vis['cluster_kmeans'] == cluster].drop('cluster_kmeans', axis=1)
    mean_profile = cluster_data.mean()
    std_profile = cluster_data.std()
    
    plt.plot(mean_profile.index, mean_profile, label=f'Кластер {cluster} (n={len(cluster_data)})')
    plt.fill_between(mean_profile.index, 
                     mean_profile - std_profile, 
                     mean_profile + std_profile, alpha=0.2)

plt.xticks(range(0, 24, 2))
plt.xlabel('Час суток')
plt.ylabel('Потребление, кВт·ч')
plt.title('Средние суточные профили потребления летом (K-means)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# =============================================
# 10. Сколько суток в каждом кластере
# =============================================
print("\nРаспределение по кластерам (K-means):")
print(pd.Series(labels_kmeans).value_counts().sort_index())

Найдено летних суток: 0
Форма данных (часы × сутки): (336, 0)


ValueError: Length mismatch: Expected axis has 336 elements, new values have 24 elements